In [1]:
from mesa import Model
from mesa import Agent
from mesa.space import MultiGrid
from mesa.time import RandomActivation
from mesa.datacollection import DataCollector
import random
import numpy as np

from a_star_pathfinding import a_star
from vision import lines
from vision import transform

In [2]:
def euclidean(pos_a, pos_b):
    return ((pos_a[0] - pos_b[0])**2 + (pos_a[1] - pos_b[1])**2)**.5

In [3]:
class AmongUs(Model):
    def __init__(self, map_name, n_crew, n_impo):
        super().__init__()

        # generate grid
        self.height = 138
        self.width = 242
        self.grid = MultiGrid(self.width, self.height, torus=True)

        # vents initialization
        self.vents_dict = {
            (125,110): (130, 110),
            (130,110): (125, 110),
            (115,110): (140, 130),
            (140,130): (115, 110)
            }
        self.vents = []

        # generate map
        self.generate_map(map_name)

        # create schedulers for agents
        self.schedule_Crewmate = RandomActivation(self)
        self.schedule_Imposter = RandomActivation(self)
        
        # initialize agents
        self.n_crew = n_crew
        self.n_impo = n_impo
        self.spawn_players()

    def generate_map(self, map_name):
        hard_walls = np.load(f'{map_name}/hardwalls.npy')
        
        for coord in hard_walls:
            coord[1] = self.height - 1 - coord[1]
            self.new_agent(Wall, tuple(coord))

        for coord in self.vents_dict:
            self.new_agent(Vent, tuple(coord))

    def spawn_players(self):
        # generate crewmates
        for _ in range(self.n_crew):
            self.new_agent(Crewmate, (125, 100))

        # generate imposters
        for _ in range(self.n_impo):
            self.new_agent(Imposter, (125, 100))

    def new_agent(self, agent_type, pos):
        '''
        Method that creates a new agent, and adds it to the correct scheduler.
        '''
        agent = agent_type(self.next_id(), self, pos)

        self.grid.place_agent(agent, pos)

        if agent_type == Crewmate or agent_type == Imposter:
            getattr(self, f'schedule_{agent_type.__name__}').add(agent)

        if agent_type == Vent:
            self.vents.append(agent)
        
    def step(self):
        '''
        Method that steps every agent. 
        
        Prevents applying step on new agents by creating a local list.
        '''

        self.schedule_Crewmate.step()
        self.schedule_Imposter.step()

        # self.datacollector.collect(self)

    def play_match(self, step_count=50):
        '''
        Method that runs the model for a specific amount of steps.
        '''
        for i in range(step_count):
            print(i)
            self.step()


In [4]:
class Crewmate(Agent):
    def __init__(self, unique_id, model, init_pos):
        super().__init__(unique_id, model)
        self.pos = init_pos
        self.path = []
        self.goal = (90, 40)

    def find_goal(self):
        self.path = a_star(self.pos, self.goal, self.model.grid)[0]

    def step(self):
        if self.path == []:
            self.find_goal()
        
        grid = self.model.grid
        neighborhood = grid.get_neighborhood(self.pos, True)
        
        neighbors = self.model.grid.get_neighbors(self.pos, True)

        if len(neighbors) > 0:
            for neighbor in neighbors:
                if type(neighbor) == Wall and neighbor.pos in neighborhood:
                    neighborhood.remove(neighbor.pos)

        # random movement
        # grid.move_agent(self, random.choice(neighborhood))

        if self.path != []:
            grid.move_agent(self, self.path.pop())
        
        self.find_vision()
        
    def find_vision(self):
        '''
        Determines the vision of the agent 
        '''
        grid = self.model.grid
        origin_lines = lines
        transformed_lines = transform(origin_lines, self.pos)
        vision_neighborhood = []
        
        for i in range(len(transformed_lines)):
            found_wall = False
            for j in range(len(transformed_lines[i])):  
                agents_on_tile = grid.get_neighbors(pos=transformed_lines[i][j], moore=True, radius=0, include_center=True)

                # stop line if it bumps into a wall  
                for agent in agents_on_tile:
                    if type(agent) == Wall:
                        found_wall = True
                
                # append to vision neigborhood when no wall is found and if coordinate is not already appended before
                if found_wall == False:
                    if transformed_lines[i][j] not in vision_neighborhood:
                        vision_neighborhood.append(transformed_lines[i][j])  
                        

class Imposter(Agent):
    def __init__(self, unique_id, model, init_pos):
        super().__init__(unique_id, model)
        self.pos = init_pos
        self.path = []
        self.goal = (90, 45)
        self.vents_enabled = True

        self.find_goal()
    
    def find_goal(self):
        best_path, best_score = a_star(self.pos, self.goal, self.model.grid)

        if self.vents_enabled:

            for vent in self.model.vents:
                vent_in = vent.pos_a
                vent_out = vent.pos_b
                # guess_vent_path = ((self.pos - vent_in)**2 + (vent_out - self.goal)**2)**.5
                guess_vent_path = euclidean(self.pos, vent_in) + euclidean(vent_out, self.goal)

                if guess_vent_path < best_score:
                    print(self.pos, vent_in)
                    vent_path_in, vent_score_in = a_star(self.pos, vent_in, self.model.grid)
                    vent_path_out, vent_score_out = a_star(vent_out, self.goal, self.model.grid)
                    vent_path = vent_path_in + vent_path_out
                    vent_score = vent_score_in + vent_score_out

                    if vent_score < best_score:
                        best_score = vent_score
                        best_path = vent_path

        self.path = best_path

        def step(self):
            if self.path == []:
                self.find_goal()

            grid = self.model.grid
            neighborhood = grid.get_neighborhood(self.pos, True)
            
            neighbors = self.model.grid.get_neighbors(self.pos, True)

            if len(neighbors) > 0:
                for neighbor in neighbors:
                    if type(neighbor) == Wall and neighbor.pos in neighborhood:
                        neighborhood.remove(neighbor.pos)

        # random movement
        # grid.move_agent(self, random.choice(neighborhood))

        if self.path != []:
            grid.move_agent(self, self.path.pop())
        
        self.find_vision()
        
    def find_vision(self):
        '''
        Determines the vision of the agent 
        '''
        grid = self.model.grid
        origin_lines = lines
        transformed_lines = transform(origin_lines, self.pos)
        vision_neighborhood = []
        
        for i in range(len(transformed_lines)):
            found_wall = False
            for j in range(len(transformed_lines[i])):  
                agents_on_tile = grid.get_neighbors(pos=transformed_lines[i][j], moore=True, radius=0, include_center=True)

                # stop line if it bumps into a wall  
                for agent in agents_on_tile:
                    if type(agent) == Wall:
                        found_wall = True
                
                # append to vision neigborhood when no wall is found and if coordinate is not already appended before
                if found_wall == False:
                    if transformed_lines[i][j] not in vision_neighborhood:
                        vision_neighborhood.append(transformed_lines[i][j])  
                        
                        
class Wall(Agent):
    def __init__(self, unique_id, model, pos):
        super().__init__(unique_id, model)
        self.pos = pos

class Obstruction(Agent):
    def __init__(self, unique_id, model, pos):
        super().__init__(unique_id, model)
        self.pos = pos

class Vent(Agent):
    def __init__(self, unique_id, model, pos):
        super().__init__(unique_id, model)
        self.pos_a = pos
        self.pos_b = self.model.vents_dict[pos]

In [5]:
amongUs = AmongUs('the_skeld', 3, 1)

amongUs.play_match()

(125, 100) (125, 110)


KeyError: (125, 110)